In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTING REQUIRED LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import math
from scipy.stats.mstats import winsorize
from scipy import stats

In [ ]:
df=pd.read_csv('/kaggle/input/life-expectancy-who/Life Expectancy Data.csv')
df.head()

# SHAPE OF DATA SET

In [ ]:
df.shape

# Columns Present in the data set

In [ ]:
df.columns

# We can see that some columns have space before and after the text, so we apply .strip to remove those spaces.

In [ ]:
df.rename(columns=lambda x: x.strip(), inplace=True)

# Structure of data set

In [ ]:
df.info()

# SUMMARY STATISTICS OF DATA SET

In [ ]:
df.describe()

# COUNT OF NULL AND ZERO VALUES

In [ ]:
def null_Zero_count(data):
    for column in data.columns.to_list():
        print('*****',column,'*****')
        print(" Number of Null values : ",data[column].isnull().sum())
        print(" Number of zeros : ",(data[column]==0).sum(axis=0))
null_Zero_count(df)

# Replacing null and zero values by mean

In [ ]:
df.mean()

In [ ]:
def null_zero_replace_mean(data):
    for column in list(data.select_dtypes(include='float64' or 'int64').columns):
        print('*****',column,'*****')
        print("*Original Count*")
        print("Zero Values: ", (data[column]==0).sum())
        print("Null Values: ", data[column].isnull().sum())
        data[column] = np.where(data[column]==0, data[column].mean(), data[column])
        data[column] = data[column].fillna(data[column].mean())
        print("*Cleaned Count*")
        print("Zero Values: ", (data[column]==0).sum())
        print("Null Values: ", data[column].isnull().sum())
    return data
null_zero_replace_mean(df)

In [ ]:
df.isnull().sum()

# Dealing with outliers

First a boxplot and histogram will be created for each continuous variable in order to visually see if outliers exist.


In [ ]:
colNames = df.columns.tolist()
for column in colNames:
    if df[column].dtype == "object": 
        continue
    plt.figure(figsize=(18,6))
    plt.subplot(1,2,1)
    plt.boxplot(df[column])
    plt.title(column)
    plt.ylabel(column)
    plt.xlabel('Bins')
    
    plt.subplot(1,2,2)
    plt.hist(df[column])
    plt.title(column)
    plt.ylabel(column)
    plt.xlabel('Bins')

# Removing outlier using winsorization

In [ ]:
for col in list(df.columns):
    if df[col].dtype == "object": 
            continue
    
    plt.figure(figsize=(18,6))

    plt.subplot(1,2,1)
    plt.boxplot(df[col])
    plt.title(col)

    plt.subplot(1,2,2)
    winsorized_data = winsorize(df[col],(0.01,0.01),inplace=True)
    plt.boxplot(winsorized_data)
    plt.title("winsorized "+col )

    plt.show()

# The range, variance and standard deviation

In [ ]:
print("****| Range |****")
print(df.select_dtypes(include='float64' or 'int64').max() -
      df.select_dtypes(include='float64' or 'int64').min())

#variance
print("****| Variance |****")
print(df.var())

#standard deviation
print("****| Standard Deviation |****")
print(df.std())

# CORRELATION 

In [ ]:
plt.figure(figsize=(16, 6))
sns.heatmap(df.corr(), annot= True)

In [ ]:
sns.pairplot(df)

# Average life expectancy from 2000 to 2015 for different countries

In [ ]:
life_expectancy_per_country = df.groupby('Country')['Life expectancy'].mean().sort_values(ascending=True)
life_expectancy_per_country.plot(kind='bar', figsize=(50,10), fontsize=12)
plt.title("Life expectancy mean per Country from 2000 to 2015",fontsize=30)
plt.xlabel("Country",fontsize=15)
plt.ylabel("Life expectancy",fontsize=15)
plt.show()

# Difference beetween life expecatancy of  developed and developing countries

In [ ]:
plt.figure(figsize=(10,10))
plt.bar(df.groupby('Status')['Status'].count().index,df.groupby('Status')['Life expectancy'].mean())
plt.xlabel("Status",fontsize=15)
plt.ylabel("Life expectancy",fontsize=15)
plt.title("Life expectancy for developed and developing country",fontsize=20)
plt.show()

# Life Expectancy in India

In [ ]:
india = df.loc[df['Country'] == "India"]
india

In [ ]:
sns.scatterplot(x=india["GDP"],y=india["Life expectancy"])
sns.lineplot(x=india["GDP"],y=india["Life expectancy"])

In [ ]:
sns.scatterplot(x=india["Year"],y=india["Life expectancy"])
sns.lineplot(x=india["Year"],y=india["Life expectancy"])

# Modeling  Year vs Life expectancy

In [ ]:
X=india['Year'].values
Y=india['Life expectancy'].values

In [ ]:
mean_x =np.mean(X)
mean_y =np.mean(Y)
l=len(X)
n=0
d=0
for i in range(l):
    n+=(X[i]-mean_x)*(Y[i]-mean_y)
    d+=(X[i]-mean_x)**2
b1=n/d
b0=mean_y-(b1*mean_x)
print(b1,b0)

In [ ]:
max_x=np.max(X)+10
min_x=np.min(X)
x=np.linspace(min_x,max_x)
y=b0+b1*x

sns.lineplot(x,y)
sns.scatterplot(x=india["Year"],y=india["Life expectancy"])

# Calculating r square

In [ ]:
ss_t=0
ss_r=0
for i in range(l):
    y_predict=b0+b1*X[i]
    ss_t +=(Y[i]-mean_y)**2
    ss_r +=(Y[i]-y_predict)**2
r2=1-(ss_r/ss_t)
print(r2)

# Predicting life expectancy of current year

In [ ]:
print(b0+b1*2021)

# Modeling GDP vs Life expectancy¶
MULTIPLE LINEAR REGRESSION

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
X=india[['Year','GDP']]
Y=india['Life expectancy']
X_train, X_test, y_train, y_test = train_test_split(X,Y)
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print('Mean squared error: ', mean_squared_error(y_test, y_pred))
print('Coefficient of determination:',r2_score(y_test, y_pred))
print("Life Expectancy - ",regr.predict([[2021, 2870]]))

# life expectancy is 70.3 years in india.